# LAB 3.1: Supervised Learning

**Temat:** Regresja i Klasyfikacja

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing, load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

np.random.seed(42)
print('✓ Biblioteki załadowane')

✓ Biblioteki załadowane


## CZĘŚĆ 1: REGRESJA

### California Housing Dataset

In [3]:
# TODO: Wczytaj California Housing dataset
# Użyj funkcji fetch_california_housing()
housing = fetch_california_housing() # TODO
X = housing.data
y = housing.target
feature_names = housing.feature_names


print(f'Shape: {X.shape}')

/home/wiktor/projects/uni/machine-learning-lab/.venv/lib/python3.12/site-packages/sklearn/datasets/_base.py:1518: UserWarning: Retry downloading from url: https://ndownloader.figshare.com/files/5976036
  warnings.warn(f"Retry downloading from url: {remote.url}")


HTTPError: HTTP Error 403: Forbidden

In [4]:
# EDA - Analiza korelacji
if X is not None:
    housing_df = pd.DataFrame(X, columns=feature_names)
    housing_df['target'] = y
    plt.figure(figsize=(10, 8))
    sns.heatmap(housing_df.corr(), annot=True, fmt='.2f', cmap='coolwarm')
    plt.title('Correlation Matrix')
    plt.show()

In [ ]:
# TODO: Podziel dane na zbiór treningowy i testowy (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# TODO: Przeskaluj dane używając StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Zadanie 1.1: Linear Regression

In [ ]:
# TODO: Zainicjalizuj i wytrenuj Linear Regression
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

# TODO: Dokonaj predykcji na zbiorze testowym
y_pred_test = lr.predict(X_test_scaled)
# Metryki
if y_pred_test is not None:
    print(f'Test MSE: {mean_squared_error(y_test, y_pred_test):.4f}')
    print(f'Test R²: {r2_score(y_test, y_pred_test):.4f}')

### Zadanie 1.2: Ridge Regression (L2)

In [6]:
# TODO: Przetestuj różne wartości alpha w pętli
alphas = [0.001, 0.01, 0.1, 1, 10, 100]
results = []

for alpha in alphas:
    # TODO: Trenuj Ridge Regression z danym alpha
    ridge = Ridge(alpha=alpha)
    ridge.fit(X_train_scaled, y_train)
    
    score = ridge.score(X_test_scaled, y_test)
    
    results.append({'alpha': alpha, 'r2': score})

results_df = pd.DataFrame(results)
print(results_df)

NameError: name 'X_train_scaled' is not defined

### Zadanie 1.3: Lasso Regression (L1)

In [5]:
# TODO: Zastosuj Lasso Regression (alpha=0.1) i sprawdź, które cechy zostały wyzerowane
lasso = Lasso(alpha=0.1)
lasso.fit(X_train_scaled, y_train)

lasso_r2 = lasso.score(X_test_scaled, y_test)
print(f'Lasso R² (alpha=0.1): {lasso_r2:.4f}')

if lasso is not None:
    zero_coef_count = np.sum(lasso.coef_ == 0)
    print(f'Liczba cech z wyzerowanym współczynnikiem: {zero_coef_count}')
    
    coef_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': lasso.coef_})
    print("Współczynniki wag dla cech:")
    print(coef_df.sort_values(by='Coefficient', ascending=False))

NameError: name 'X_train_scaled' is not defined

## CZĘŚĆ 2: KLASYFIKACJA

### Breast Cancer Dataset

In [7]:
# TODO: Wczytaj dane Breast Cancer
breast_cancer = load_breast_cancer()
X_bc = breast_cancer.data
y_bc = breast_cancer.target

# TODO: Podziel na train/test (stratify=y_bc) i przeskaluj dane
X_train_bc, X_test_bc, y_train_bc, y_test_bc = train_test_split(
    X_bc, y_bc, test_size=0.2, stratify=y_bc, random_state=42
)

scaler_bc = StandardScaler()
X_train_bc_scaled = scaler_bc.fit_transform(X_train_bc)
X_test_bc_scaled = scaler_bc.transform(X_test_bc)

### Zadanie 2.1: Porównanie Modeli

In [8]:
# TODO: Zdefiniuj słownik z modelami: LogisticRegression, DecisionTree, RandomForest, SVM, KNN
models = {
    'Logistic Regression': LogisticRegression(),
    'Decision Tree': DecisionTreeClassifier(max_depth=5, random_state=42),
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(kernel='linear', probability=True),
    'KNN': KNeighborsClassifier(n_neighbors=5)
}

# Pętla trenująca i oceniająca
results_bc = []

for name, model in models.items():
    model.fit(X_train_bc_scaled, y_train_bc)
    
    y_pred = model.predict(X_test_bc_scaled)
    
    acc = accuracy_score(y_test_bc, y_pred)
    results_bc.append({'Model': name, 'Accuracy': acc})
    
    print(f"{name} Accuracy: {acc:.4f}")

results_df = pd.DataFrame(results_bc).sort_values(by='Accuracy', ascending=False)
print("Ranking modeli:")
print(results_df)

Logistic Regression Accuracy: 0.9825
Decision Tree Accuracy: 0.9211
Random Forest Accuracy: 0.9561
SVM Accuracy: 0.9737
KNN Accuracy: 0.9561
Ranking modeli:
                 Model  Accuracy
0  Logistic Regression  0.982456
3                  SVM  0.973684
2        Random Forest  0.956140
4                  KNN  0.956140
1        Decision Tree  0.921053


Wnioski: na tych labolatoriach poznaliśmy klasyczne algorytmy uczenia nadzorowanego w odniesieniu do regresji i klasyfikacji. Kolejny raz najważniejszy jest preprocessing danych oraz dużą rolę odegrała reularyzacja (Lasso, Ridge) w prewencji przed przeuczeniem modelu. Przy porównaniu modeli wyszło, że potężne modele jak np. Random Forest wypadają gorzej od zwykłej regresji logistycznej, co jest spowodowane małym rozmiarem datasetu.